In [2]:
import pandas as pd
from sqlalchemy import create_engine, text

In [3]:
engine = create_engine('mysql+mysqlconnector://root:@localhost/m5-forecast')

with engine.connect() as connection:
    connection.execute(text("ROLLBACK"))

In [32]:
data = pd.read_csv('./sales_train_sql.csv')

In [34]:
data = data[['store_id', 'state_id']]
data = data.drop_duplicates().reset_index(drop=True)
data

,store_id,state_id
0,CA_1,CA
1,TX_1,TX
2,WI_1,WI


In [30]:
data = data[['item_id', 'dept_id', 'cat_id']]
data = data.drop_duplicates().reset_index(drop=True)
data

,item_id,dept_id,cat_id
0,HOBBIES_1_001,HOBBIES_1,HOBBIES
1,HOBBIES_1_002,HOBBIES_1,HOBBIES
2,HOBBIES_1_003,HOBBIES_1,HOBBIES
3,HOBBIES_1_004,HOBBIES_1,HOBBIES
4,HOBBIES_1_005,HOBBIES_1,HOBBIES
5,HOBBIES_1_006,HOBBIES_1,HOBBIES
6,HOBBIES_1_007,HOBBIES_1,HOBBIES
7,HOBBIES_1_008,HOBBIES_1,HOBBIES
8,HOBBIES_1_009,HOBBIES_1,HOBBIES
9,HOBBIES_1_010,HOBBIES_1,HOBBIES


In [4]:
id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

# 2. Transform from Wide to Long
# This will turn d_1, d_2... into rows under a 'day' column
data_long = data.melt(
    id_vars=id_vars, 
    var_name='day', 
    value_name='sales_units'
)

# 3. (Optional) Convert 'day' to a cleaner integer format
# 'd_1' becomes 1
data_long['day'] = data_long['day'].str.replace('d_', '').astype(int)

In [35]:
data.to_sql(
    'stores',  
    con=engine, 
    if_exists='append', 
    index=False, 
    chunksize=10000,  # Sends 10,000 rows at a time
    method='multi'    # More efficient for many databases (like PostgreSQL/MySQL)
)

3

In [17]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Configuración
n_products = 50 
n_stores = 3
start_date = datetime(2026, 1, 1)
days_in_month = 31

# 1. GENERAR CALENDAR.CSV ACTUALIZADO
calendar_data = []
for i in range(days_in_month):
    date = start_date + timedelta(days=i)
    
    # Simulación de eventos
    event1_n, event1_t = np.nan, np.nan
    event2_n, event2_t = np.nan, np.nan
    
    if i == 0: # 1 de enero
        event1_n, event1_t = 'NewYear', 'National'
    elif i == 18: # Ejemplo de coincidencia (MLK Day + Evento ficticio)
        event1_n, event1_t = 'MartinLutherKingDay', 'National'
        event2_n, event2_t = 'OrthodoxChristmas', 'Religious'
    
    calendar_data.append({
        'date': date.strftime('%Y-%m-%d'),
        'wm_yr_wk': 11600 + (date.isocalendar()[1]),
        'weekday': date.strftime('%A'),
        'wday': (date.weekday() + 2) % 7 or 7,
        'month': date.month,
        'year': date.year,
        'd': f'd_{i+1}',
        'event_name_1': event1_n,
        'event_type_1': event1_t,
        'event_name_2': event2_n,
        'event_type_2': event2_t,
        'snap_CA': np.random.choice([0, 1]),
        'snap_TX': np.random.choice([0, 1]),
        'snap_WI': np.random.choice([0, 1])
    })
df_calendar = pd.DataFrame(calendar_data)

# 2. GENERAR SALES_TRAIN_MELTED.CSV (Mantiene estructura previa)
sales_data = []
product_ids = [f'HOBBIES_1_{str(i).zfill(3)}' for i in range(1, n_products + 1)]
stores = ['CA_1', 'TX_1', 'WI_1']

for store in stores:
    state = store.split('_')[0]
    for prod in product_ids:
        full_id = f'{prod}_{store}_validation'
        for i in range(days_in_month):
            sales_data.append({
                'id': full_id, 'item_id': prod, 'dept_id': 'HOBBIES_1',
                'cat_id': 'HOBBIES', 'store_id': store, 'state_id': state,
                'd': f'd_{i+1}', 'sales': np.random.poisson(lam=1)
            })
df_sales_melted = pd.DataFrame(sales_data).iloc[:10000]

# 3. GENERAR SELL_PRICES.CSV
price_data = []
for store in stores:
    for prod in product_ids:
        base_price = np.random.uniform(2, 20)
        for week in df_calendar['wm_yr_wk'].unique():
            price_data.append({
                'store_id': store, 'item_id': prod, 'wm_yr_wk': week,
                'sell_price': round(base_price, 2)
            })
df_prices = pd.DataFrame(price_data).iloc[:10000]

# Guardar
df_calendar.to_csv('calendar_sql.csv', index=False)
df_sales_melted.to_csv('sales_train_sql.csv', index=False)
df_prices.to_csv('sell_prices_sql.csv', index=False)

print("Archivos generados con las columnas de eventos adicionales.")

Archivos generados con las columnas de eventos adicionales.
